In [58]:
# imports - usual suspects
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# for distance and h-clustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist, squareform

# sklearn does have some functionality too, but mostly a wrapper to scipy
from sklearn.metrics import pairwise_distances 
from sklearn.preprocessing import StandardScaler

In [59]:
SQL = "SELECT * from `questrom.SMSspam.train`"
YOUR_BILLING_PROJECT = "ba820-329602"
train_spam = pd.read_gbq(SQL, YOUR_BILLING_PROJECT)

In [60]:
SQL = "SELECT * from `questrom.SMSspam.test`"
YOUR_BILLING_PROJECT = "ba820-329602"
test_spam = pd.read_gbq(SQL, YOUR_BILLING_PROJECT)

In [61]:
train_spam.duplicated().sum()

0

In [62]:
train_spam.isna().sum()

message    0
label      0
id         0
dtype: int64

In [63]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer  

In [64]:
cv = CountVectorizer()
cv.fit(train_spam.message)
dtm = cv.transform(train_spam.message)
dtm_df = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names())
dtm_df.columns

Index(['00', '000', '000pes', '008704050406', '0089', '0121', '01223585334',
       '0125698789', '02', '0207',
       ...
       'zealand', 'zed', 'zindgi', 'zoe', 'zogtorius', 'zouk', 'zyada', 'èn',
       'ú1', '〨ud'],
      dtype='object', length=6875)

In [65]:
X = dtm_df.copy()
y = train_spam.label      

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=820, stratify=y)

In [67]:
tree = DecisionTreeClassifier(max_depth=5, min_samples_split=30, min_samples_leaf=15)
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=15, min_samples_split=30)

In [68]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder

In [69]:
pipe_ord = make_pipeline(
    RandomForestClassifier(n_estimators=100, random_state=10, max_depth=14, oob_score=True, n_jobs=-1, criterion="gini", min_samples_split=5, max_features=6)
)

pipe_ord.fit(X_train, y_train)
print(pipe_ord.score(X, y))

C:\ProgramData\Anaconda3\envs\ba820\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


0.8461538461538461


In [70]:
test_spam

,message,id
0,Even my brother is not like to speak with me. ...,4
1,WINNER!! As a valued network customer you have...,5
2,Eh u remember how 2 spell his name... Yes i di...,11
3,Lol your always so convincing.,19
4,K tell me anything about you.,21
...,...,...
422,Love has one law; Make happy the person you lo...,4206
423,I want to be inside you every night...,4212
424,Miles and smiles r made frm same letters but d...,4220
425,What about this one then.,4231


In [71]:
dtm = cv.transform(test_spam.message)
dtm_df = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names())
dtm_df.columns

C:\ProgramData\Anaconda3\envs\ba820\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Index(['00', '000', '000pes', '008704050406', '0089', '0121', '01223585334',
       '0125698789', '02', '0207',
       ...
       'zealand', 'zed', 'zindgi', 'zoe', 'zogtorius', 'zouk', 'zyada', 'èn',
       'ú1', '〨ud'],
      dtype='object', length=6875)

In [72]:
X = dtm_df.copy() 

In [75]:
pred = tree.predict(X)

In [80]:
pred = pd.Series(pred)

In [82]:
submission = pd.concat([test_spam['id'], pred], axis=1)

In [83]:
submission.to_csv("./submission.csv",index=False)